<a href="https://colab.research.google.com/github/monda00/horse-race-notebook/blob/master/horse_race_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# netkeibaでスクレイピングしたデータのEDA

- 準備
- ライブラリ・データ読み込み
- データの概観・分析

# 準備

# ライブラリ・データ読み込み

In [1]:
import numpy as np
import pandas as pd

In [2]:
DATA_PATH = './'

In [6]:
race_df = pd.read_csv(DATA_PATH + 'race.csv')
horse_df = pd.read_csv(DATA_PATH + 'horse.csv')

In [7]:
race_df.head()

,clockwise,date,distance,field_condition,field_type,name,place,race_id,race_round,start_time,weather
0,右,2020/6/22,1400,不,ダ,タイタン賞競走,大井,202044062212,12R,20:50,雨
1,右,2020/6/22,1800,不,ダ,ポートサイド賞競走,大井,202044062211,11R,20:10,雨
2,右,2020/6/22,1200,不,ダ,C1九 十11,大井,202044062210,10R,19:30,雨
3,右,2020/6/22,1400,不,ダ,C1九 十11,大井,202044062209,9R,18:55,雨
4,右,2020/6/22,1600,不,ダ,C1九 十11,大井,202044062208,8R,18:20,雨


In [8]:
horse_df.head()

,agari,age,frame_number,horse_number,horse_weight,jockey,name,popular,race_date,race_id,race_name,rank,time,weight,win
0,38.1,牡3,7.0,13,511(-5),森泰斗,ワイルドホース,2.0,2020/6/22,202044062212,タイタン賞競走,1.0,1:28.5,54.0,3.7
1,38.1,牝5,6.0,11,475(-2),山崎誠士,クインズオライリー,12.0,2020/6/22,202044062212,タイタン賞競走,2.0,1:28.7,54.0,81.4
2,38.6,セ6,3.0,4,469(+2),西啓太,メダーリアフレイム,4.0,2020/6/22,202044062212,タイタン賞競走,3.0,1:28.8,56.0,8.6
3,37.9,牡4,8.0,14,526(+6),藤本現暉,ゴールドプリンス,8.0,2020/6/22,202044062212,タイタン賞競走,4.0,1:28.8,56.0,34.4
4,38.5,牝5,4.0,7,475(+6),藤田凌,イグレット,9.0,2020/6/22,202044062212,タイタン賞競走,5.0,1:28.9,54.0,34.8


# データの概観・分析

In [9]:
race_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27532 entries, 0 to 27531
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   clockwise        27532 non-null  object
 1   date             27532 non-null  object
 2   distance         27532 non-null  int64 
 3   field_condition  21830 non-null  object
 4   field_type       21830 non-null  object
 5   name             27532 non-null  object
 6   place            27532 non-null  object
 7   race_id          27532 non-null  object
 8   race_round       27532 non-null  object
 9   start_time       26755 non-null  object
 10  weather          26690 non-null  object
dtypes: int64(1), object(10)
memory usage: 2.3+ MB


In [10]:
horse_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100093 entries, 0 to 100092
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   agari         90608 non-null   float64
 1   age           100093 non-null  object 
 2   frame_number  99678 non-null   float64
 3   horse_number  100093 non-null  int64  
 4   horse_weight  100093 non-null  object 
 5   jockey        99959 non-null   object 
 6   name          100092 non-null  object 
 7   popular       99158 non-null   float64
 8   race_date     100092 non-null  object 
 9   race_id       100092 non-null  object 
 10  race_name     100092 non-null  object 
 11  rank          100092 non-null  float64
 12  time          98439 non-null   object 
 13  weight        100092 non-null  float64
 14  win           100092 non-null  object 
dtypes: float64(5), int64(1), object(9)
memory usage: 11.5+ MB


## データ数

In [13]:
print('race:', race_df.shape[0])
print('horse:', horse_df.shape[0])

race: 27532
horse: 100093


## カラム

In [17]:
race_df.columns

Index(['clockwise', 'date', 'distance', 'field_condition', 'field_type',
       'name', 'place', 'race_id', 'race_round', 'start_time', 'weather'],
      dtype='object')

In [18]:
horse_df.columns

Index(['agari', 'age', 'frame_number', 'horse_number', 'horse_weight',
       'jockey', 'name', 'popular', 'race_date', 'race_id', 'race_name',
       'rank', 'time', 'weight', 'win'],
      dtype='object')

## 欠損値の確認

In [14]:
race_df.isnull().sum()

clockwise             0
date                  0
distance              0
field_condition    5702
field_type         5702
name                  0
place                 0
race_id               0
race_round            0
start_time          777
weather             842
dtype: int64

In [15]:
horse_df.isnull().sum()

agari           9485
age                0
frame_number     415
horse_number       0
horse_weight       0
jockey           134
name               1
popular          935
race_date          1
race_id            1
race_name          1
rank               1
time            1654
weight             1
win                1
dtype: int64